In [1]:
import duckdb
import pandas as pd
import json

con = duckdb.connect('bluesky_180MB.duckdb')

collections_df = con.execute("""
    SELECT 
        collection,
        COUNT(*) as count,
        ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM records), 2) as percentage
    FROM records 
    GROUP BY collection 
    ORDER BY count DESC 
    LIMIT 10
""").fetchdf()

total_rows = con.execute("SELECT COUNT(*) as total FROM records").fetchdf().iloc[0,0]
print(f"\nTotal records: {total_rows:,}")
print("\nTop 10 collections by record count:")
display(collections_df)


Total records: 900,000

Top 10 collections by record count:


,collection,count,percentage
0,app.bsky.feed.like,453558,50.40
1,app.bsky.graph.follow,205106,22.79
2,app.bsky.feed.repost,122556,13.62
3,app.bsky.feed.post,96415,10.71
4,app.bsky.graph.block,11719,1.30
5,app.bsky.graph.listitem,4926,0.55
6,app.bsky.actor.profile,4055,0.45
7,chat.bsky.actor.declaration,634,0.07
8,app.bsky.graph.listblock,448,0.05
9,app.bsky.feed.postgate,272,0.03


In [2]:
con.execute("""
    WITH parsed_records AS (
        SELECT 
            *,
            JSON_EXTRACT_STRING(record, '$.createdAt') as profile_created_at
        FROM records 
        WHERE collection = 'app.bsky.actor.profile'
    )
    SELECT
        MIN(profile_created_at) as earliest_profile,
        MAX(profile_created_at) as latest_profile
    FROM parsed_records
""").fetchdf()


,earliest_profile,latest_profile
0,2024-02-07T20:08:11.868Z,2025-01-08T17:59:24.403Z


In [3]:
# Query one random example from each collection
for collection_name in collections_df['collection'].head(7):
    query = f"""
    SELECT repo, rkey, at_rev, record
    FROM records 
    WHERE collection = '{collection_name}'
    ORDER BY RANDOM()
    LIMIT 1
    """
    
    result = con.execute(query).fetchdf()
    
    if not result.empty:
        # Extract all fields
        record = json.loads(result['record'][0])
        repo = result['repo'][0]
        rkey = result['rkey'][0]
        at_rev = result['at_rev'][0]
        
        filename = f"examples/{collection_name.replace('.', '_')}.json"
        
        # Combine all fields into one JSON object
        json_data = {
            'repo': repo,
            'rkey': rkey,
            'at_rev': at_rev,
            'record': record
        }
        
        with open(filename, 'w') as f:
            json.dump(json_data, f, indent=2)
        
        print(f"Saved example for {collection_name} to {filename}")

Saved example for app.bsky.feed.like to examples/app_bsky_feed_like.json
Saved example for app.bsky.graph.follow to examples/app_bsky_graph_follow.json
Saved example for app.bsky.feed.repost to examples/app_bsky_feed_repost.json
Saved example for app.bsky.feed.post to examples/app_bsky_feed_post.json
Saved example for app.bsky.graph.block to examples/app_bsky_graph_block.json
Saved example for app.bsky.graph.listitem to examples/app_bsky_graph_listitem.json
Saved example for app.bsky.actor.profile to examples/app_bsky_actor_profile.json


In [33]:
# Example of creating user-post interaction matrix
interaction_query = """
WITH user_interactions AS (
    -- Likes
    SELECT 
        repo as user_id,
        TRY_CAST(JSON_EXTRACT_STRING(record, '$.subject.uri') AS VARCHAR) as post_uri,
        'like' as interaction_type,
        TRY_CAST(JSON_EXTRACT_STRING(record, '$.createdAt') AS TIMESTAMP) as timestamp
    FROM records
    WHERE collection = 'app.bsky.feed.like'
        AND record IS NOT NULL
        AND TRY_CAST(record AS JSON) IS NOT NULL  -- Skip invalid JSON
    
    UNION ALL
    
    -- Posts
    SELECT 
        repo as user_id,
        'at://' || repo || '/app.bsky.feed.post/' || rkey as post_uri,
        'create' as interaction_type,
        TRY_CAST(JSON_EXTRACT_STRING(record, '$.createdAt') AS TIMESTAMP) as timestamp
    FROM records
    WHERE collection = 'app.bsky.feed.post'
        AND record IS NOT NULL
        AND TRY_CAST(record AS JSON) IS NOT NULL  -- Skip invalid JSON
)
SELECT 
    user_id,
    post_uri,
    interaction_type,
    timestamp
FROM user_interactions
WHERE post_uri IS NOT NULL
    AND timestamp IS NOT NULL
LIMIT 10  -- Reduced limit for testing
"""

interaction_df = con.execute(interaction_query).fetchdf()
print("Shape:", interaction_df.shape)
display(interaction_df.head())

InvalidInputException: Invalid Input Error: Malformed JSON at byte 0 of input: input length is 0.  Input: 

In [ ]:
query = "SELECT * FROM "